# Amplifier Support

Work out the data structures to support the detectors (not area detectors) and amplifiers used by the USAXS.

Basic setup.

In [1]:
from collections import OrderedDict
import epics
import logging
import threading
import time

from ophyd import Device, Component, EpicsSignal, EpicsSignalRO, EpicsScaler
from ophyd import FormattedComponent, DynamicDeviceComponent
import bluesky
import bluesky.plans
import databroker

FORMAT = '%(asctime)s %(levelname)s %(name)s - %(message)s'
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO, format=FORMAT)

RE = bluesky.RunEngine({})
db = databroker.Broker.named("mongodb_config")
RE.subscribe(db.insert)
RE.msg_hook = bluesky.utils.ts_msg_hook

## simple amplifier device

In [2]:
class CurrentAmplifierDevice(Device):
    gain = Component(EpicsSignalRO, "gain")


class FemtoAmplifierDevice(CurrentAmplifierDevice):
    gainindex = Component(EpicsSignal, "gainidx")
    description = Component(EpicsSignal, "femtodesc")

### Table of detectors and related support

```
========  =================  ====================  ===================  ===========
detector  scaler             amplifier             sequence             Femto model
========  =================  ====================  ===================  ===========
UPD       9idcLAX:vsc:c0.S4  9idcLAX:fem01:seq01:  9idcLAX:pd01:seq01:  DLPCA200
UPD       9idcLAX:vsc:c0.S4  9idcLAX:fem09:seq02:  9idcLAX:pd01:seq02:  DDPCA300
I0        9idcLAX:vsc:c0.S2  9idcRIO:fem02:seq01:  9idcLAX:pd02:seq01:
I00       9idcLAX:vsc:c0.S3  9idcRIO:fem03:seq01:  9idcLAX:pd03:seq01:
I000      9idcLAX:vsc:c0.S6  9idcRIO:fem04:seq01:  None
TRD       9idcLAX:vsc:c0.S5  9idcRIO:fem05:seq01:  9idcLAX:pd05:seq01:
========  =================  ====================  ===================  ===========
```

A PV (``9idcLAX:femto:model``) tells which UPD amplifier and sequence 
programs we're using now.  This PV is read-only since it is set when 
IOC boots, based on a soft link that configures the IOC.  The soft 
link may be changed using the ``use200pd``  or  ``use300pd`` script.

We only need to get this once, get it via one-time call with PyEpics
and then use it with inline dictionaries to pick the right PVs.


In [3]:
_amplifier_id_upd = epics.caget("9idcLAX:femto:model", as_string=True)
print("UPD amplifier: ", _amplifier_id_upd)

UPD amplifier:  DDPCA300


In [4]:
UPD_femto  = FemtoAmplifierDevice(
    dict(
        DLPCA200 = "9idcLAX:fem01:seq01:",
        DDPCA300 = "9idcLAX:fem09:seq02:",
    )[_amplifier_id_upd],
    name='upd_femto')

In [5]:
while not UPD_femto.connected:
    time.sleep(0.1)
UPD_femto.read()

OrderedDict([('upd_femto_gain',
              {'timestamp': 1527005502.683289, 'value': 1000000000000.0}),
             ('upd_femto_gainindex',
              {'timestamp': 1527005502.683289, 'value': 8}),
             ('upd_femto_description',
              {'timestamp': 631152000.0, 'value': 'New Femto300 PD'})])

## scaler channel

In [6]:
scaler0 = EpicsScaler('9idcLAX:vsc:c0', name='scaler0')

In [7]:
scaler0.read()

OrderedDict([('scaler0_channels_chan10',
              {'timestamp': 1524941203.16494, 'value': 0.0}),
             ('scaler0_channels_chan15',
              {'timestamp': 1524941203.16494, 'value': 0.0}),
             ('scaler0_channels_chan18',
              {'timestamp': 1524941203.16494, 'value': 0.0}),
             ('scaler0_channels_chan7',
              {'timestamp': 1524941203.16494, 'value': 0.0}),
             ('scaler0_channels_chan21',
              {'timestamp': 1524941203.16494, 'value': 0.0}),
             ('scaler0_channels_chan17',
              {'timestamp': 1524941203.16494, 'value': 0.0}),
             ('scaler0_channels_chan16',
              {'timestamp': 1524941203.16494, 'value': 0.0}),
             ('scaler0_channels_chan12',
              {'timestamp': 1524941203.16494, 'value': 0.0}),
             ('scaler0_channels_chan4',
              {'timestamp': 1524941203.16494, 'value': 104374.0}),
             ('scaler0_channels_chan8',
              {'timestamp': 15

That's too many channels.  Let's find out (and reconfigure ophyd to read only) the *interesting* channels (the ones with names assigned by users).

In [8]:
def use_EPICS_scaler_channels(scaler):
    """
    configure scaler for only the channels with names assigned in EPICS 
    """
    read_attrs = []
    for ch in scaler.channels.component_names:
        _nam = epics.caget("{}.NM{}".format(scaler.prefix, int(ch[4:])))
        if len(_nam.strip()) > 0:
            read_attrs.append(ch)
            print("{}.S{}".format(scaler.prefix,int(ch[4:])-1), ch, _nam)
    scaler.channels.read_attrs = read_attrs

use_EPICS_scaler_channels(scaler0)
scaler0.read()

9idcLAX:vsc:c0.S0 chan1 seconds
9idcLAX:vsc:c0.S1 chan2 I0_USAXS
9idcLAX:vsc:c0.S2 chan3 I00_USAXS
9idcLAX:vsc:c0.S3 chan4 PD_USAXS
9idcLAX:vsc:c0.S4 chan5 TR diode
9idcLAX:vsc:c0.S5 chan6 I000


OrderedDict([('scaler0_channels_chan1',
              {'timestamp': 1524941203.16494, 'value': 10000000.0}),
             ('scaler0_channels_chan2',
              {'timestamp': 1524941203.16494, 'value': 30456.0}),
             ('scaler0_channels_chan3',
              {'timestamp': 1524941203.16494, 'value': 31630.0}),
             ('scaler0_channels_chan4',
              {'timestamp': 1524941203.16494, 'value': 104374.0}),
             ('scaler0_channels_chan5',
              {'timestamp': 1524941203.16494, 'value': 1.0}),
             ('scaler0_channels_chan6',
              {'timestamp': 1524941203.16494, 'value': 0.0}),
             ('scaler0_time', {'timestamp': 1524941203.16494, 'value': 1.0})])

## Autorange support

Most of these detector amplifiers have sequence program support to adjust the gain automatically.  The programs are all built from the same model, allowing the choice of five different possible choices for gain.

The software operates in three modes: *automatic*, *auto+background*, and *manual*.

In [9]:
class AutorangeSettings(object):
    """values allowed for sequence program's ``reqrange`` PV"""
    automatic = "automatic"
    auto_background = "auto+background"
    manual = "manual"

A first attempt to provide for the sequence program support defines the gain (but makes it harder to interface if the choices are different than what has been programmed).

In [10]:
class DiodeRangeDevice(Device):
    _default_configuration_attrs = ()
    _default_read_attrs = ('gain', 'background', 'background_error')

    gain = FormattedComponent(EpicsSignal, '{self.prefix}gain{self._ch_num}')
    background = FormattedComponent(EpicsSignal, '{self.prefix}bkg{self._ch_num}')
    background_error = FormattedComponent(EpicsSignal, '{self.prefix}bkgErr{self._ch_num}')

    def __init__(self, prefix, ch_num=None, **kwargs):
        assert ch_num is not None, "Must provide `ch_num=` keyword argument."
        self._ch_num = ch_num
        super().__init__(prefix, **kwargs)


class AmplifierSequenceControlsDevice(CurrentAmplifierDevice):
    """
    Ophyd support for amplifier sequence program
    """
    reqrange = Component(EpicsSignal, "reqrange")
    mode = Component(EpicsSignal, "mode")
    selected = Component(EpicsSignal, "selected")
    gainU = Component(EpicsSignal, "gainU")
    gainD = Component(EpicsSignal, "gainD")
    range0 = Component(DiodeRangeDevice, '', ch_num=0)
    range1 = Component(DiodeRangeDevice, '', ch_num=1)
    range2 = Component(DiodeRangeDevice, '', ch_num=2)
    range3 = Component(DiodeRangeDevice, '', ch_num=3)
    range4 = Component(DiodeRangeDevice, '', ch_num=4)
    counts_per_volt = Component(EpicsSignal, "vfc")
    status = Component(EpicsSignalRO, "updating")
    lurange = Component(EpicsSignalRO, "lurange")
    lucounts = Component(EpicsSignalRO, "lucounts")
    lurate = Component(EpicsSignalRO, "lurate")
    lucurrent = Component(EpicsSignalRO, "lucurrent")
    updating = Component(EpicsSignalRO, "updating")

    def __init__(self, prefix, **kwargs):
        self.scaler = None
        super().__init__(prefix, **kwargs)

    def measure_dark_currents(self, numReadings=8):     # TODO: part of #16
        """
        """
        assert self.scaler is not None, "Must define the `scaler`."
        pass

    def autoscale(self):                                # TODO: #16
        """
        """
        assert self.scaler is not None, "Must define the `scaler`."
        pass

    @property
    def isUpdating(self):
        v = self.mode.value in (1, AutorangeSettings.auto_background)
        if v:
            v = self.updating.value in (1, "Updating")
        return v

    def _decode_gain_target(self, target):
        """
        returns gain setting from requested ``target`` value

        Should override in subclass to customize for different 
        amplifiers.  These are for USAXS photodiode.

        PARAMETERS

        target : int
            one of (4, 6, 8, 10, 12)
            corresponding, respectively, to gains of (1e4, 1e6, 1e8, 1e10, 1e12)


        EXAMPLE CONTENT::

            gain_list = (4, 6, 8, 10, 12)
            err_msg = "supplied value ({}) not one of these: {}".format(
                target, gain_list)
            assert target in gain_list, err_msg
            return gain_list.index(target)

        """
        raise NotImplementedError("Must define in subclass")

    def set_gain_plan(self, target):
        """
        set gain on amplifier during a BlueSky plan

        Only use low noise gains; those are the only ones which actually work
        """
        yield from bps.abs_set(self.mode, AutorangeSettings.manual)
        yield from bps.abs_set(self.reqrange, self._decode_gain_target(target))

    def set_gain_cmd(self, target):
        """
        set gain on amplifier directly, do not use during a BlueSky plan

        Only use low noise gains; those are the only ones which actually work
        """
        self.mode.put(AutorangeSettings.manual)
        self.reqrange.put(self._decode_gain_target(target))

That defines some generic support.  Now we create subclasses to define the specific ranges and how to set them.

In [11]:
class UPD_AmplifierSequenceControlsDevice(AmplifierSequenceControlsDevice):

    def _decode_gain_target(self, target):
        """
        converts ``target`` value to gain (index) value for USAXS photodiode

        PARAMETERS

        target : int
            one of (4, 6, 8, 10, 12)
            corresponding, respectively, to gains of (1e4, 1e6, 1e8, 1e10, 1e12)

        """
        gain_list = (4, 6, 8, 10, 12)
        err_msg = "`target` must be one of these values: {}".format(gain_list)
        assert target in gain_list, err_msg
        return gain_list.index(target)


class TRD_AmplifierSequenceControlsDevice(AmplifierSequenceControlsDevice):

    def _decode_gain_target(self, target):
        gain_list = (5, 6, 7, 8, 9, 10)
        err_msg = "`target` must be one of these values: {}".format(gain_list)
        assert target in gain_list, err_msg
        if target < 10:
            fmt = "1e{} low noise"
        else:
            fmt = "1e{} high speed"
        return fmt.format(target)


Connect that structure with the sequence controls for the UPD detector.

In [12]:
UPD_seq = UPD_AmplifierSequenceControlsDevice(
    dict(
        DLPCA200 = "9idcLAX:pd01:seq01:",
        DDPCA300 = "9idcLAX:pd01:seq02:",
    )[_amplifier_id_upd],
    name="UPD_seq")

In [13]:
while not UPD_seq.connected:
    time.sleep(0.1)
UPD_seq.read()

OrderedDict([('UPD_seq_gain',
              {'timestamp': 1527084381.286855, 'value': 1000000000000.0}),
             ('UPD_seq_reqrange',
              {'timestamp': 1527084381.286855, 'value': 4}),
             ('UPD_seq_mode', {'timestamp': 1526924030.29391, 'value': 2}),
             ('UPD_seq_selected', {'timestamp': 631152000.0, 'value': 0}),
             ('UPD_seq_gainU',
              {'timestamp': 1524632376.675726, 'value': 4000.0}),
             ('UPD_seq_gainD',
              {'timestamp': 1524632376.675726, 'value': 650000.0}),
             ('UPD_seq_range0_gain',
              {'timestamp': 631152000.0, 'value': 10000.0}),
             ('UPD_seq_range0_background',
              {'timestamp': 1524630657.499278, 'value': 5.0}),
             ('UPD_seq_range0_background_error',
              {'timestamp': 1524630657.499278, 'value': 0.0}),
             ('UPD_seq_range1_gain',
              {'timestamp': 631152000.0, 'value': 1000000.0}),
             ('UPD_seq_range1_backgro

-----------

# Questions about this support before we continue

1. Since the sequence program knows (from st.cmd) to which amplifier and scaler it is using (and this configuration is not available in any EPICS PVs), must we need coordinate that here as well?
1. Can we create a device so that we pass the custom ranges as arguments to the constructor?
1. Should the number of gains be created dynamically?
1. Is there a problem with creating more than one ophyd connection to any PV?

## Answers

1. Seems like a good idea.  Make one device with all related signals. Build on previously-defined structures, if possible.  Needs a custom constructor since three PVs will be passed in the setup.
2. Add that to the new device.  Use `ophyd.DynamicDeviceComponent` similar to `ophyd.EpicsScaler`.
   **BUT**, the ranges may be obtained from the sequence program's `reqrange` `enum_strs`!
3. Like above.  **BUT**, this has to match what is provided by the EPICS database.
4. Don't know.  Good question for Tom Caswell.

# Round 2: Combined scaler, amplifier, autorange controls device

First, show how to setup the class

In [14]:
class DetectorAmplifierAutorangeDevice(Device):
    
    def __init__(self, autorange_pv, scaler_channel_pv, amplifier_pv, ranges, **kwargs):
        self.autorange_pv = autorange_pv
        self.scaler_channel_pv = scaler_channel_pv
        self.amplifier_pv = amplifier_pv
        self.ranges = ranges
        super().__init__(autorange_pv, **kwargs)

upd_struct = DetectorAmplifierAutorangeDevice(
    "9idcLAX:pd01:seq02:",
    "9idcLAX:vsc:c0.S4",
    "9idcLAX:fem09:seq02:",
    {"1e4":4, "1e6":6, "1e8":8, "1e10":10, "1e12":12},
    name="upd_struct",
)

In [15]:
upd_struct.read()

OrderedDict()

To allow for a dynamic number of ranges, we need to treat them as a subgroup.  Redefine previous `AmplifierSequenceControlsDevice`.

In [16]:
def _ranges_subgroup_(cls, nm, suffix, ranges, **kwargs):
    defn = OrderedDict()
    for i in ranges:
        key = '{}{}'.format(nm, i)
        defn[key] = (cls, '', dict(ch_num=i))

    return defn


class AmplifierSequenceControlsDevice(CurrentAmplifierDevice):
    """
    Ophyd support for amplifier sequence program
    """
    reqrange = Component(EpicsSignal, "reqrange")
    mode = Component(EpicsSignal, "mode")
    selected = Component(EpicsSignal, "selected")
    gainU = Component(EpicsSignal, "gainU")
    gainD = Component(EpicsSignal, "gainD")
    ranges = DynamicDeviceComponent(
        _ranges_subgroup_(
            DiodeRangeDevice, 'range', 'suffix', range(5)))
    counts_per_volt = Component(EpicsSignal, "vfc")
    status = Component(EpicsSignalRO, "updating")
    lurange = Component(EpicsSignalRO, "lurange")
    lucounts = Component(EpicsSignalRO, "lucounts")
    lurate = Component(EpicsSignalRO, "lurate")
    lucurrent = Component(EpicsSignalRO, "lucurrent")
    updating = Component(EpicsSignalRO, "updating")

    def __init__(self, prefix, **kwargs):
        self.scaler = None
        super().__init__(prefix, **kwargs)

    def measure_dark_currents(self, numReadings=8):     # TODO: part of #16
        """
        """
        assert self.scaler is not None, "Must define the `scaler`."
        pass

    def autoscale(self):                                # TODO: #16
        """
        """
        assert self.scaler is not None, "Must define the `scaler`."
        pass

    @property
    def isUpdating(self):
        v = self.mode.value in (1, AutorangeSettings.auto_background)
        if v:
            v = self.updating.value in (1, "Updating")
        return v

    def _decode_gain_target(self, target):
        """
        returns gain setting from requested ``target`` value

        Should override in subclass to customize for different 
        amplifiers.  These are for USAXS photodiode.

        PARAMETERS

        target : int
            one of (4, 6, 8, 10, 12)
            corresponding, respectively, to gains of (1e4, 1e6, 1e8, 1e10, 1e12)


        EXAMPLE CONTENT::

            gain_list = (4, 6, 8, 10, 12)
            err_msg = "supplied value ({}) not one of these: {}".format(
                target, gain_list)
            assert target in gain_list, err_msg
            return gain_list.index(target)

        """
        raise NotImplementedError("Must define in subclass")

    def set_gain_plan(self, target):
        """
        set gain on amplifier during a BlueSky plan

        Only use low noise gains; those are the only ones which actually work
        """
        yield from bps.abs_set(self.mode, AutorangeSettings.manual)
        yield from bps.abs_set(self.reqrange, self._decode_gain_target(target))

    def set_gain_cmd(self, target):
        """
        set gain on amplifier directly, do not use during a BlueSky plan

        Only use low noise gains; those are the only ones which actually work
        """
        self.mode.put(AutorangeSettings.manual)
        self.reqrange.put(self._decode_gain_target(target))

Next, connect up the existing structures

In [17]:
class DetectorAmplifierAutorangeDevice(Device):
    scaler = FormattedComponent(EpicsSignal, '{self.scaler_channel_pv}')
    femto = FormattedComponent(FemtoAmplifierDevice, '{self.amplifier_pv}')
    controls = FormattedComponent(AmplifierSequenceControlsDevice, '{self.autorange_pv}')
    
    def __init__(self, autorange_pv, scaler_channel_pv, amplifier_pv, range_dict, **kwargs):
        self.autorange_pv = autorange_pv
        self.scaler_channel_pv = scaler_channel_pv
        self.amplifier_pv = amplifier_pv
        self.range_dict = range_dict
        super().__init__("", **kwargs)

upd_struct = DetectorAmplifierAutorangeDevice(
    "9idcLAX:pd01:seq02:",
    "9idcLAX:vsc:c0.S4",
    "9idcLAX:fem09:seq02:",
    {"1e4":4, "1e6":6, "1e8":8, "1e10":10, "1e12":12},
    name="upd_struct",
)

In [18]:
upd_struct.read()

OrderedDict([('upd_struct_scaler',
              {'timestamp': 1524941203.16494, 'value': 104374.0}),
             ('upd_struct_femto_gain',
              {'timestamp': 1527005502.683289, 'value': 1000000000000.0}),
             ('upd_struct_femto_gainindex',
              {'timestamp': 1527005502.683289, 'value': 8}),
             ('upd_struct_femto_description',
              {'timestamp': 631152000.0, 'value': 'New Femto300 PD'}),
             ('upd_struct_controls_gain',
              {'timestamp': 1527084381.570188, 'value': 1000000000000.0}),
             ('upd_struct_controls_reqrange',
              {'timestamp': 1527084381.570188, 'value': 4}),
             ('upd_struct_controls_mode',
              {'timestamp': 1526924030.29391, 'value': 2}),
             ('upd_struct_controls_selected',
              {'timestamp': 631152000.0, 'value': 0}),
             ('upd_struct_controls_gainU',
              {'timestamp': 1524632376.675726, 'value': 4000.0}),
             ('upd_struct_

In [19]:
upd_struct.summary()

data keys (* hints)
-------------------
 upd_struct_controls_counts_per_volt
 upd_struct_controls_gain
 upd_struct_controls_gainD
 upd_struct_controls_gainU
 upd_struct_controls_lucounts
 upd_struct_controls_lucurrent
 upd_struct_controls_lurange
 upd_struct_controls_lurate
 upd_struct_controls_mode
 upd_struct_controls_ranges_range0_background
 upd_struct_controls_ranges_range0_background_error
 upd_struct_controls_ranges_range0_gain
 upd_struct_controls_ranges_range1_background
 upd_struct_controls_ranges_range1_background_error
 upd_struct_controls_ranges_range1_gain
 upd_struct_controls_ranges_range2_background
 upd_struct_controls_ranges_range2_background_error
 upd_struct_controls_ranges_range2_gain
 upd_struct_controls_ranges_range3_background
 upd_struct_controls_ranges_range3_background_error
 upd_struct_controls_ranges_range3_gain
 upd_struct_controls_ranges_range4_background
 upd_struct_controls_ranges_range4_background_error
 upd_struct_controls_ranges_range4_gain
 upd_stru

# Set gain on the Femto amplifier

We'd like to add a method to set the gain on the Femto amplifiers.  Since the gain values are available from EPICS, we can use that to provide a method that will set the gain by integer index number, by desired gain floating-point value, or by gain text value.

In [22]:
upd_struct.femto.gainindex.enum_strs

('1e4 V/A',
 '1e5 V/A',
 '1e6 V/A',
 '1e7 V/A',
 '1e8 V/A',
 '1e9 V/A',
 '1e10 V/A',
 '1e11 V/A',
 '1e12 V/A',
 '1e13 V/A',
 'UNDEF',
 'UNDEF',
 'UNDEF',
 'UNDEF',
 'UNDEF',
 'UNDEF')

Looks easy to create a list of all acceptable values.  Like this:

In [24]:
def showme(enum_strs):
    return [s for s in enum_strs if s != 'UNDEF']

showme(upd_struct.femto.gainindex.enum_strs)

['1e4 V/A',
 '1e5 V/A',
 '1e6 V/A',
 '1e7 V/A',
 '1e8 V/A',
 '1e9 V/A',
 '1e10 V/A',
 '1e11 V/A',
 '1e12 V/A',
 '1e13 V/A']

Good starting point.  Let's return all acceptable values as targets for a new `setGain()` method.

In [27]:
def showme(enum_strs):
    acceptable = [s for s in enum_strs if s != 'UNDEF']
    num_gains = len(acceptable)
    # assume labels are ALWAYS formatted: "{float} V/A"
    acceptable += [float(s.split()[0]) for s in acceptable]
    acceptable += range(num_gains)
    return acceptable

showme(upd_struct.femto.gainindex.enum_strs)

['1e4 V/A',
 '1e5 V/A',
 '1e6 V/A',
 '1e7 V/A',
 '1e8 V/A',
 '1e9 V/A',
 '1e10 V/A',
 '1e11 V/A',
 '1e12 V/A',
 '1e13 V/A',
 10000.0,
 100000.0,
 1000000.0,
 10000000.0,
 100000000.0,
 1000000000.0,
 10000000000.0,
 100000000000.0,
 1000000000000.0,
 10000000000000.0,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9]

Good.  Now, add to the Femto-specific amplifier support:

In [60]:
class FemtoAmplifierDevice(CurrentAmplifierDevice):
    gainindex = Component(EpicsSignal, "gainidx")
    description = Component(EpicsSignal, "femtodesc")
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._range_info_known = False
        self.num_ranges = 0
        self.acceptable_range_values = ()
        
    def __init_range_info__(self, enum_strs):
        acceptable = [s for s in enum_strs if s != 'UNDEF']
        num_gains = len(acceptable)
        # assume labels are ALWAYS formatted: "{float} V/A"
        acceptable += [float(s.split()[0]) for s in acceptable]
        acceptable += range(num_gains)
        self.num_ranges = num_gains
        self.acceptable_range_values = acceptable

    def setGain(self, target):
        if not self._range_info_known:
            self.__init_range_info__(self.gainindex.enum_strs)
        if target in self.acceptable_range_values:
            if isinstance(target, (int, float)) and target > self.num_ranges:
                # gain value specified, rewrite as str
                # assume mantissa is only 1 digit
                target = ("%.0e V/A" % target).replace("+", "")
            self.gainindex.put(target)
        else:
            msg = "could not set gain to {}, ".format(target)
            msg += "must be one of these: {}".format(self.gainindex.enum_strs)
            raise ValueError(msg)

while not femto.connected:
    time.sleep(0.1)
femto = FemtoAmplifierDevice("9idcLAX:fem09:seq02:", name="femto")
print(femto.num_ranges, "\n", femto.acceptable_range_values)

0 
 ()


In [61]:
import math
math.log10(femto.gain.value)

8.0

In [62]:
femto.setGain("1e7 V/A")
math.log10(femto.gain.value)

7.0

In [65]:
femto.setGain(4)
print(math.log10(femto.gain.value), femto.gainindex.enum_strs[4])

8.0 1e8 V/A


In [67]:
femto.setGain(1e12)
math.log10(femto.gain.value)

12.0